# Data Collection

In [ ]:
import gym
import cv2
import numpy as np
import random
import os

In [ ]:

SAVE_FREQUENCY = 50 # Save 3 consecutive frames every nth step



# Convert color to grayscale
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

# Turn discrete scalar action into continuous
def format_action(num):
  if num == 0:
    action = [-1, 0, 0]
  elif num == 1:
    action = [1, 0, 0]
  elif num == 2:
    action = [0, 1, 0]
  elif num == 3:
    action = [0, 0, 1]
  else:
    raise Exception()
  
  return action

# Simplify image (B&W, round values)
def process_image(img):
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  img = rgb2gray(img)
  img = np.round(img/100)*50
  return img

# Save image
def save_picture(img, round, i):
  cv2.imwrite('/content/imgs/img_{}_{}.png'.format(round, i), img)

env = gym.make("CarRacing-v2")

round = 0

while True:
  done = False
  observation = env.reset()
  i = 0
  # Complete 200 steps
  while not done and i<200:
    action = env.action_space.sample() # Random action
    action = format_action(action)
    observation, reward, done, info = env.step(action)
    # Stack frames if multiple of 50
    if i >= SAVE_FREQUENCY and i%SAVE_FREQUENCY == 0:
      images = np.zeros((96, 96, 3))
      images[:, :, 0] = process_image(observation)
    elif i >= SAVE_FREQUENCY and i%SAVE_FREQUENCY == 1:
      images[:, :, 1] = process_image(observation)
    elif i >= SAVE_FREQUENCY and i%SAVE_FREQUENCY == 2:
      # Save images if third frame
      images[:, :, 2] = process_image(observation)
      if random.random()<.5:
        images = cv2.flip(images, 1)
      save_picture(images, round, i)
    i += 1
  print('Done round {}'.format(round))
  round += 1

env.close()

In [ ]:
# Import cv2 and numpy
import cv2
import numpy as np
import os

folder = "" # Images stored folder

images = []

# Loop through the files in the folder
for file in sorted(os.listdir(folder)):
    img = cv2.imread(folder + "/" + file)

    images.append(img)

images = np.array(images)

images = images/150 # Normalizing
print(images.shape)

# Training

In [ ]:
import cv2
from os import walk
import numpy as np
import random

import keras
from keras import layers
from keras.layers import Reshape, Flatten, Conv2D, Dense, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.optimizers import Adam, RMSprop, Adadelta

In [ ]:
encoder_input = keras.Input(shape=(96, 96, 3))

x = Conv2D(16, (3, 3), padding='same', strides=2)(encoder_input)
x = Conv2D(8, (3, 3), padding='same', strides=2)(x)
x = Conv2D(4, (5, 5), padding='same', strides=2)(x)
x = Conv2D(2, (5, 5), padding='same', strides=2)(x)
x = Conv2D(1, (5, 5), padding='same')(x)

x = Reshape((36,))(x)
encoder_output = Dense(32, activation='sigmoid')(x)

encoder = keras.Model(encoder_input, x)
encoder.summary()

In [ ]:
# decoder_input = keras.Input(shape=(16,))

decoder_output = Dense(32)(encoder_output)
decoder_output = Dense(72)(decoder_output)
decoder_output = Reshape((6, 6, 2))(decoder_output)
decoder_output = Conv2DTranspose(2, (7, 7), padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2DTranspose(4, (5, 5), padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2DTranspose(8, (3, 3), padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2DTranspose(16, (3, 3), padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(decoder_output)

# Define the decoder model object
auto_encoder = keras.Model(encoder_input, decoder_output)

# Define the auto-encoder model by combining the encoder and decoder
auto_encoder.summary()



In [ ]:
auto_encoder.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=.01))
# auto_encoder.compile(loss='mse', optimizer=Adadelta())

auto_encoder.summary()

In [ ]:
auto_encoder.fit(
    x=images,
    y=images,
    batch_size=32,
    epochs=300,
    verbose="auto",
    validation_split=0.2)

# Testing

In [ ]:
from google.colab.patches import cv2_imshow
test_image = cv2.imread('/content/imgs/img_132_102.png')
cv2_imshow(test_image)
test_image = np.reshape(test_image, (1, 96, 96, 3))/150

In [ ]:
output = auto_encoder.predict(test_image)
output = np.reshape(output, (96, 96, 3))*150
cv2_imshow(output)

In [ ]:
cv2_imshow(np.clip(output[..., 0] - output[..., 2], 0, 10)*20)

In [ ]:
cv2_imshow(np.clip(output[..., 0] - output[..., 2], -10, 0)*-20)

# Saving Encoder

In [ ]:
save_encoder_input = keras.layers.Input((96, 96, 3))
save_encoder = save_encoder_input
for layer in auto_encoder.layers[1:8]:
  print(layer, layer.name)
  save_encoder = layer(save_encoder)

save_encoder = keras.models.Model(inputs=save_encoder_input, outputs=save_encoder)
save_encoder.summary()